In [3]:
options(tidyverse.quiet = TRUE)
library(tidyverse)
library(janitor)
library(lubridate)
library(PlayerRatings)
source("../src/utils.R")

In [6]:
# Read in all rae results and one-vs-one race results
# 2009
# Tongyeong
ty_df_09 <- read_csv("../data/2009_races/races/Tongyeong_men.csv", col_types = cols())
ty_ovo_09 <- read_csv("../data/2009_races/ovo_races/Tongyeong_men.csv", col_types = cols())
# Madrid
md_df_09 <- read_csv("../data/2009_races/races/Madrid_men.csv", col_types = cols())
md_ovo_09 <- read_csv("../data/2009_races/ovo_races/Madrid_men.csv", col_types = cols())
# Washington
ws_df_09 <- read_csv("../data/2009_races/races/Washington_men.csv", col_types = cols())
ws_ovo_09 <- read_csv("../data/2009_races/ovo_races/Washington_men.csv", col_types = cols())
# Kitz
kz_df_09 <- read_csv("../data/2009_races/races/Kitz_men.csv", col_types = cols())
kz_ovo_09 <- read_csv("../data/2009_races/ovo_races/Kitz_men.csv", col_types = cols())
# Hamburg
hb_df_09 <- read_csv("../data/2009_races/races/Hamburg_men.csv", col_types = cols())
hb_ovo_09 <- read_csv("../data/2009_races/ovo_races/Hamburg_men.csv", col_types = cols())
# London
ld_df_09 <- read_csv("../data/2009_races/races/London_men.csv", col_types = cols())
ld_ovo_09 <- read_csv("../data/2009_races/ovo_races/London_men.csv", col_types = cols())
# Yokohama
yk_df_09 <- read_csv("../data/2009_races/races/Yokohama_men.csv", col_types = cols())
yk_ovo_09 <- read_csv("../data/2009_races/ovo_races/Yokohama_men.csv", col_types = cols())
# Gold Coast
gc_df_09 <- read_csv("../data/2009_races/races/Gold-coast_men.csv", col_types = cols())
gc_ovo_09 <- read_csv("../data/2009_races/ovo_races/Gold-coast_men.csv", col_types = cols())

# 2010
# Sydney
sd_df_10 <- read_csv("../data/2010_races/races/Sydney_men.csv", col_types = cols())
sd_ovo_10 <- read_csv("../data/2010_races/ovo_races/Sydney_men.csv", col_types = cols())
# Seoul
sl_df_10 <- read_csv("../data/2010_races/races/Seoul_men.csv", col_types = cols())
sl_ovo_10 <- read_csv("../data/2010_races/ovo_races/Seoul_men.csv", col_types = cols())
# Madrid
md_df_10 <- read_csv("../data/2010_races/races/Madrid_men.csv", col_types = cols())
md_ovo_10 <- read_csv("../data/2010_races/ovo_races/Madrid_men.csv", col_types = cols())
# Hamburg
hb_df_10 <- read_csv("../data/2010_races/races/Hamburg_men.csv", col_types = cols())
hb_ovo_10 <- read_csv("../data/2010_races/ovo_races/Hamburg_men.csv", col_types = cols())
# London
ld_df_10 <- read_csv("../data/2010_races/races/London_men.csv", col_types = cols())
ld_ovo_10 <- read_csv("../data/2010_races/ovo_races/London_men.csv", col_types = cols())
# Kitz
kz_df_10 <- read_csv("../data/2010_races/races/Kitz_men.csv", col_types = cols())
kz_ovo_10 <- read_csv("../data/2010_races/ovo_races/Kitz_men.csv", col_types = cols())
# Budapest
bd_df_10 <- read_csv("../data/2010_races/races/Budapest_men.csv", col_types = cols())
bd_ovo_10 <- read_csv("../data/2010_races/ovo_races/Budapest_men.csv", col_types = cols())



# List of races
races <- list(list(ty_df_09, ty_ovo_09), list(ws_df_09, ws_ovo_09), list(kz_df_09, kz_ovo_09),
              list(hb_df_09, hb_ovo_09), list(ld_df_09, ld_ovo_09), list(yk_df_09, yk_ovo_09), 
              list(gc_df_09, gc_ovo_09), list(sd_df_10, sd_ovo_10), list(sl_df_10, sl_ovo_10), 
              list(md_df_10, md_ovo_10), list(hb_df_10, hb_ovo_10), list(ld_df_10, ld_ovo_10), 
              list(kz_df_10, kz_ovo_10), list(bd_df_10, bd_ovo_10))




In [9]:
### Find optimal level of C
c_vec <- seq(1, 5, by=1)

# Period incrementer for races
average_results <- rep(NA, length(c_vec))
iterator <- 1

# DF to store results at different levels of c
# iterate through each c
for (c in c_vec[1]){
    # reset period
    period <- 1
    # reset glicko and correlations
    glicko <- glicko_ratings(ty_ovo_09)
    rankings_df <- updated_rankings(glicko)
    # This can be a vector as above
    correlation_tracker <- rep(NA, length(races))
    # iterate through each race
    for (race in races){
        race_df <- race[[1]]
        race_ovo <- race[[2]]
        # adjust period for the glicko race objects
        period <- period + 1
        # pass in rotating value of c
        update <- race_update(glicko, rankings_df, race_df, race_ovo, c)
        # save results from the update for use in next iteration
        glicko <- update[[1]]
        rankings_df <- update[[2]]
        correlation_tracker[period-1] <- update[[3]]
    }
    # Find the average kendall's tau at that level of C
    average_results[iterator] <- correlation_tracker %>% mean()
    iterator <- iterator + 1
}
results_summary <- tibble(c = c_vec, average_kendall_tau = average_results)
results_summary

ERROR: Error in cor(race_gl$glicko_rank, race_gl$position, method = "kendall"): 'y' must be numeric


In [10]:
for (race in races[1]){
        race_df <- race[[1]]
        race_ovo <- race[[2]]
        # adjust period for the glicko race objects
        period <- period + 1
        # pass in rotating value of c
        update <- race_update(glicko, rankings_df, race_df, race_ovo, c)
        # save results from the update for use in next iteration
        glicko <- update[[1]]
        rankings_df <- update[[2]]
        correlation_tracker[period-1] <- update[[3]]
    }

In [13]:
glicko

Player,Rating,Deviation,Games,Win,Draw,Loss,Lag
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7788,2357.324,46.96140,174,174,0,0,1
5695,2288.843,45.72091,163,158,0,5,1
5692,2169.439,40.13147,165,149,0,16,1
5332,2155.657,38.46503,213,191,0,22,1
5747,2120.748,28.67894,371,341,0,30,0
5296,2096.211,27.16011,424,382,0,42,0
7747,2091.882,27.94728,389,351,0,38,0
5662,2080.390,26.90689,404,360,0,44,0
6442,2065.891,28.91457,360,320,0,40,0
